# Visualize Generated Videos

This notebook provides some utilites to visualize the generated videos along with their prompts. 

In [ ]:
import json
import os
import random
from pathlib import Path

import mediapy
from dotenv import load_dotenv

# Load the environment variables
load_dotenv()
MODEL_ROOT = Path(os.getenv("MODEL_PATH"))
DATA_ROOT = Path(os.getenv("DATA_PATH"))


def visualize_video(frame_paths: list[Path], **kwargs):
    images = [mediapy.read_image(frame_path) for frame_path in sorted(frame_paths)]
    mediapy.show_video(images, **kwargs)

def visualize_instance(
    model_name: str,
    motion: str | None = None,
    visual_style: str | None = None,
    scene_type: str | None = None,
    layout: str | None = None,
    instance_id: str | None = None,
):

    result_dir = MODEL_ROOT / model_name / "worldscore_output"

    # Iterate over different levels of result categorization. Order matters.
    for _type in [motion, visual_style, scene_type, layout, instance_id]:
        possible_options = [x.name for x in result_dir.iterdir()]
        if _type is None:
            _type = random.choice(possible_options)
        elif _type not in possible_options:
            raise ValueError(f"Option {_type} not in {possible_options}")
        
        result_dir = result_dir / _type
    
    
    # Load camera data
    with open(result_dir / "image_data.json") as json_f:
        data_dict = json.load(json_f)

    if "image" in data_dict:
        prompt_image_path = DATA_ROOT / data_dict["image"]
        prompt_image = mediapy.read_image(prompt_image_path)
        mediapy.show_image(prompt_image, title=data_dict["image"], height=512)

    frames = sorted([x for x in result_dir.glob("frames/*.png")])
    visualize_video(frames, loop=False, height=512, fps=)

    print("---- Prompt Options ----")
    for key, value in data_dict.items():
        print(f"{key}: {value}")

In [ ]:
visualize_instance(model_name="VideoCrafter1_t2v", motion="static", visual_style="photorealistic", scene_type="indoor", layout="dining_spaces", instance_id="043")